# State

In [55]:
newtype State s a = 
  State {
    runState ::
      s -> (a,s) 
  }
  
mkState :: (s->(a,s)) -> State s a
mkState f = State {runState = f}

In [4]:
st :: State Int String 
st = State {runState = \s -> ("value", s+1)}

In [5]:
runState st 2

("value",3)

## Functor

In [37]:
instance Functor (State s) where
  --fmap ::
  --  (a -> b) -> State s a -> State s b
  fmap f sa = 
       State (\s -> let (a, s') = runState sa s in (f a,s'))

In [7]:
let sa = State(\s -> ("value", s+1))
let sb = fmap (\v -> v ++ "...") sa

In [8]:
runState sb 0

("value...",1)

In [76]:
times3 :: (a->a) -> (a->a)
times3 f = f . f . f
addDots :: String -> String
addDots s = s ++ "..."

In [44]:
let sc = addDots <$> sa
runState sc 5

("value...",6)

## Applicative

In [9]:
instance Applicative (State s) where
  -- pure :: a -> State s a
    pure x = State(\s -> (x,s))
  -- <*> State s (a -> b) -> State s a -> State s b
    (<*>) sab sa = State (\s -> 
                    let (ab,s') = runState sab s
                        (a,s'') = runState sa s'
                    in (ab a, s''))

Line 3: Use tuple-section
Found:
\ s -> (x, s)
Why not:
(x,)

In [67]:
let ss = State (\s -> (addDots,s)) <*> State (\s -> (addDashes,s+1)) 
runState ss 0

("value...",1)

In [78]:
let ss2 = State (\s -> (times3,s)) <*> State (\s -> (addDots,s+1)) <*> State (\s -> ("value",s+1)) 
runState ss2 0

("value.........",2)

## Monad

In [10]:
instance Monad (State s) where
    -- return :: a -> State s a
    return = pure
    -- (=<<) :: (a -> State s b) -> State s a -> State s b
    --(=<<) = flip (>>=)
    -- (>>=) :: State s a -> (a -> State s b) -> State s b
    (>>=) sa f = State(\s -> let (a,s') = runState sa s
                             in runState (f a) s')
    

In [148]:
let addArrow = (\v -> State (\s -> (v ++ " -> ",s + 1)))
let addDots = (\v -> State (\s -> (v ++ " ... ",s + 1)))
let c = State (\s -> ("value", s+1)) >>= addArrow >>= addArrow
runState c 1

("value ->  -> ",4)

In [205]:
let cc = do 
    x <- addDots
    y <- addArrow
    let z = join (++) x y--State (\s -> let (a,s') = runState x s
--                              (b,s'') = runState y s'
--                              in (a ++ b, s''))
    return z
runState (State (\s -> ("value", s+1)) >>= cc) 0

("value ... value -> ",3)

## Exec

In [110]:
exec :: State s a -> s -> s
exec sa s = snd (runState sa s)

Line 2: Use execState
Found:
snd (runState sa s)
Why not:
execState sa sLine 2: Use execState
Found:
snd (runState sa s)
Why not:
execState sa s

## Eval

In [111]:
eval :: State s a -> s -> a
eval sa s = fst (runState sa s)

Line 2: Use evalState
Found:
fst (runState sa s)
Why not:
evalState sa sLine 2: Use evalState
Found:
fst (runState sa s)
Why not:
evalState sa s

## Get, Put

In [195]:
get :: State s s
get = State(\s -> (s,s))
put :: s -> State s ()
put s = State(\_ -> ((),s))

Line 4: Use const
Found:
\ _ -> ((), s)
Why not:
const ((), s)

In [202]:
runState get 1

(1,1)

## Join states

In [203]:
join :: (a->b->c) -> State s a -> State s b -> State s c
join f sa sb = State (\s -> let (a, s') = runState sa s
                                (b, s'') = runState sb s'
                            in (f a b, s''))

In [16]:
instance Semigroup a => Semigroup (State s a) where
  (<>) sa1 sa2 = State (\s -> let (a1, s1) = runState sa1 s
                                  (a2, s2) = runState sa2 s1
                              in (a1 <> a2, s2))


In [53]:
instance Monoid a => Monoid (State s a) where
  mempty = State(\s -> (mempty, s))
  mappend = (<>)
  mconcat = foldr mappend mempty

Line 2: Use tuple-section
Found:
\ s -> (mempty, s)
Why not:
(mempty,)

In [40]:
let s1 = State(\s -> ("1->",s))
let s2 = State(\s -> ("2->",s+1))
let s3 = State(\s -> ("3",s+2))
runState (fmap reverse (s1 <> s2 <> s3)) 4

("3>-2>-1",7)

In [41]:
runState (mconcat [s1, s2, s3]) 3

("1->2->3",6)

In [73]:
:info Traversable

type Traversable :: (* -> *) -> Constraint
class (Functor t, Foldable t) => Traversable t where
  traverse :: Applicative f => (a -> f b) -> t a -> f (t b)
  sequenceA :: Applicative f => t (f a) -> f (t a)
  mapM :: Monad m => (a -> m b) -> t a -> m (t b)
  sequence :: Monad m => t (m a) -> m (t a)
  {-# MINIMAL traverse | sequenceA #-}
  	-- Defined in ‘Data.Traversable’
instance Traversable [] -- Defined in ‘Data.Traversable’
instance Traversable Maybe -- Defined in ‘Data.Traversable’
instance Traversable (Either a) -- Defined in ‘Data.Traversable’
instance Traversable ((,) a) -- Defined in ‘Data.Traversable’

In [74]:
sequence [Just 1, Just 3]

Just [1,3]

In [77]:
words "1,2"

["1,2"]